In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append('../src')

In [3]:
import torch
import scipy.sparse as sp
import glob
import warnings
import anndata
import matplotlib.pyplot as plt
import scanpy as sc
import numpy as np
import seaborn as sns
import torch

In [4]:
import spaceoracle
from spaceoracle import SpaceOracle

In [5]:
adata_train = sc.read_h5ad(
    '/ihome/ylee/kor11/space/SpaceOracle/notebooks/cache/adata_train.h5ad')

OMP: Info #276: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.


In [1]:
# import shutil
# import os
    
# models_dir = './models'
# if os.path.exists(models_dir):
#     shutil.rmtree(models_dir)
# os.makedirs(models_dir)

In [16]:
adata_train

AnnData object with n_obs × n_vars = 9043 × 1725
    obs: 'cluster', 'rctd_cluster', 'n_genes_by_counts', 'log1p_n_genes_by_counts', 'total_counts', 'log1p_total_counts', 'pct_counts_in_top_50_genes', 'pct_counts_in_top_100_genes', 'pct_counts_in_top_200_genes', 'pct_counts_in_top_500_genes', 'total_counts_mt', 'log1p_total_counts_mt', 'pct_counts_mt', 'n_counts'
    var: 'mt', 'n_cells_by_counts', 'mean_counts', 'log1p_mean_counts', 'pct_dropout_by_counts', 'total_counts', 'log1p_total_counts', 'n_cells', 'highly_variable', 'means', 'dispersions', 'dispersions_norm'
    uns: 'cluster_colors', 'rctd_cluster_colors', 'log1p', 'hvg'
    obsm: 'X_spatial', 'rctd_results', 'spatial'
    layers: 'raw_count', 'normalized_count', 'imputed_count'

In [11]:
so = spaceoracle.SpaceOracle(
    adata=adata_train,
    annot='rctd_cluster', 
    max_epochs=5, 
    learning_rate=4e-4, 
    spatial_dim=64,
    batch_size=256,
    init_betas='co',
    rotate_maps=True,
    cluster_grn=True,
    regularize=True,
)

🌍️ Generating spatial maps: 100%|██████████| 9043/9043 [00:13<00:00, 690.17it/s]


CPU times: user 1min 45s, sys: 17.9 s, total: 2min 3s
Wall time: 39.2 s


In [50]:
so.run()

Training interrupted...


AttributeError: 'NoneType' object has no attribute 'eval'